In [ ]:
!nvidia-smi -L

In [ ]:
!pip install transformers --upgrade
!pip install datasets
!pip install torch
!pip install torchtext
!add-apt-repository -y ppa: jonathonf/ffmpegia
!apt update
!apt install -y ffmpeg

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install pandas

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install torch -U

In [ ]:
from dataclasses import dataclass
from typing import List, Optional, Union, Any, Dict, List, Union
import torch
from google.colab import userdata
import pandas as pd

In [ ]:
from huggingface_hub import login

access_token_read = userdata.get('HuggingFace')
login(token = access_token_read)
print("Logged into Hugging Face")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Dataset can be found at: https://drive.google.com/drive/folders/1fHfq0v5f3vgomyWPhVdD9WlDbSqpsREp?usp=sharing


In [ ]:
import os

local_tsv_path = os.chdir('/content/drive/MyDrive/ja')
df = pd.read_csv("covost_v2.ja_en.tsv", sep='\t')

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

common_voice = DatasetDict()

common_voice['train'] = Dataset.from_pandas(train_df)
common_voice['test'] = Dataset.from_pandas(test_df)

common_voice = common_voice.remove_columns(["split", '__index_level_0__'])
print(common_voice)

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [ ]:
print(f'{common_voice["train"][2]}\n')

In [ ]:
print(os.getcwd())

In [ ]:
import torchaudio
from torchaudio.transforms import Resample

def resample_audio(audio_path):
    waveform, original_sampling_rate = torchaudio.load(audio_path)

    resample = Resample(original_sampling_rate, 16000)
    resampled_waveform = resample(waveform)

    return resampled_waveform.numpy(), 16000

def prepare_dataset(batch):
    audio_path = "/content/drive/MyDrive/ja/clips/" + batch["path"]
    resampled_audio, new_sampling_rate = resample_audio(audio_path)

    batch["audio"] = {"array": resampled_audio, "sampling_rate": new_sampling_rate}

    batch["input_features"] = feature_extractor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(batch["translation"]).input_ids

    return batch

common_voice = common_voice.map(
    prepare_dataset,
    num_proc=2
)

In [ ]:
common_voice["test"] = common_voice["test"].remove_columns(["path"])
common_voice["train"] = common_voice["train"].remove_columns(["path"])

print(common_voice)

In [ ]:
from typing import Any, Dict, List, Union
import torch
from dataclasses import dataclass

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-finetuned-common-voice-en",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=250,
    max_steps=(len(common_voice["train"])// 16) * 2,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=(len(common_voice["train"])//16) * 2,
    eval_steps=(len(common_voice["train"])//16) * 2,
    logging_steps=5,
    report_to=["tensorboard"],
    load_best_model_at_end=False,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    remove_unused_columns=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


In [ ]:
try:
    print('Start training...')
    trainer.train()
    print('Training is finished.')
    trainer.save_model("./whisper-finetuned-common-voice-ja-en")
    tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
    tokenizer.save_pretrained("./whisper-finetuned-common-voice-ja-en")
except Exception as e:
    print(f'Error during training: {e}')
